# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df =pd.read_csv('../WeatherPy/output_data/output_data_cities.csv')
# weather_df

# Remove extra index
del weather_df['Unnamed: 0']

weather_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,sinnamary,GF,5.38,-52.95,84.45,67,4,9.01,1607964205
1,vaini,TO,-21.20,-175.20,78.80,88,75,10.29,1607964205
2,cape town,ZA,-33.93,18.42,71.01,72,75,12.75,1607964205
3,avarua,CK,-21.21,-159.78,77.00,88,100,4.70,1607964206
4,berbera,SO,10.44,45.01,77.41,58,11,10.25,1607964206
...,...,...,...,...,...,...,...,...,...
548,tambacounda,SN,13.75,-13.76,98.60,12,0,4.70,1607964299
549,saint-joseph,RE,-21.37,55.62,77.00,94,90,17.22,1607964299
550,havre-saint-pierre,CA,50.23,-63.60,28.40,86,90,13.87,1607964299
551,kalianget,ID,-7.05,113.93,80.47,80,100,7.38,1607964300


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in location, and humidity as humidity
location = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)

In [5]:
# Create heat map of all cities based on humidity.

# Define the heatmap parameters
fig=gmaps.figure(center = [0,0] ,zoom_level = 2)

# Create and add heat layer 
heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                               dissipating=False, max_intensity=200,
                               point_radius = 4)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# New DataFrame Narrow down the cities to fit weather conditions.

ideal_weather_df = weather_df[weather_df['Max Temp'] <=80]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Max Temp'] >=70]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Wind Speed'] <=10]
ideal_weather_df = ideal_weather_df[ideal_weather_df['Cloudiness'] ==0]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df = ideal_weather_df.reset_index()

ideal_weather_df

,index,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,36,chuy,UY,-33.70,-53.46,77.49,32,0,3.94,1607964211
1,146,salalah,OM,17.02,54.09,77.00,33,0,9.17,1607964109
2,373,bilma,NE,18.69,12.92,73.63,24,0,9.95,1607964271
3,434,akyab,MM,20.15,92.90,72.57,73,0,9.37,1607964281
4,462,puri,IN,19.80,85.85,73.40,88,0,2.24,1607964286


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = ideal_weather_df

# Add column for Hotel Name
hotel_df['Hotel Name'] = ""
hotel_df['Hotel Address'] = ''
hotel_df = hotel_df.drop(columns='index')
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Hotel Address
0,chuy,UY,-33.70,-53.46,77.49,32,0,3.94,1607964211,,
1,salalah,OM,17.02,54.09,77.00,33,0,9.17,1607964109,,
2,bilma,NE,18.69,12.92,73.63,24,0,9.95,1607964271,,
3,akyab,MM,20.15,92.90,72.57,73,0,9.37,1607964281,,
4,puri,IN,19.80,85.85,73.40,88,0,2.24,1607964286,,


In [8]:
# Params dictionary to update each iteration for hotels with 5000 meters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    #get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Hotel Address
0,chuy,UY,-33.70,-53.46,77.49,32,0,3.94,1607964211,Nuevo Hotel Plaza,"Avenida General Artigas, Chuy"
1,salalah,OM,17.02,54.09,77.00,33,0,9.17,1607964109,HAMDAN PLAZA HOTEL,Salalah
2,bilma,NE,18.69,12.92,73.63,24,0,9.95,1607964271,Expeditions Ténére voyages,Bilma
3,akyab,MM,20.15,92.90,72.57,73,0,9.37,1607964281,Hotel Memory,"19, Akaut Yone Street, Kyae Oin Gyi Quarter, S..."
4,puri,IN,19.80,85.85,73.40,88,0,2.24,1607964286,OYO 1577 Hotel Shree Hari Grand,"Marchikot Chhaka, Grand Road, Puri"


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
fig=gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer(location, weights=humidity,
                               dissipating=False, max_intensity=200,
                               point_radius = 4)

hotel_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,info_box_content=hotel_info)

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))